In [19]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import nibabel as nib

def create_3d_cnn(input_shape,learning_rate,beta1,beta2):
    model = Sequential()
    
    model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))

    # Create optimizer with custom LR
    optimizer = Adam(learning_rate=learning_rate, beta_1=beta1,beta_2=beta2)

    # Compile model with it
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    
    return model

input_shape = (64, 64, 64, 1)

In [20]:
import os
import numpy as np

# Load the NIfTI image
directory = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/'
files = os.listdir(directory)
files2 = []
for i in files:
    if 'nii' in i:
        files2.append(i)

import numpy as np
from scipy.ndimage import zoom

def resize_3d_image(image, target_shape=None, scale=None, order=1):
    if scale is None and target_shape is None:
        raise ValueError("Either target_shape or scale must be provided.")

    if scale is None:
        # Compute scale factors from target shape
        scale = [t / s for t, s in zip(target_shape, image.shape[:3])]

    # Handle channels
    if image.ndim == 4:
        resized = zoom(image, scale + [1], order=order)  # Keep channels unchanged
    else:
        resized = zoom(image, scale, order=order)

    return resized


image_dict = {}
count = 0
for j in files2:
    if j != '.nii.gz' and count < 300:
        path1 = directory + j
        nifti_image = nib.load(path1)
        image_data = nifti_image.get_fdata()
        max_value = np.max(image_data)
        image_data = image_data / max_value
        image_data = resize_3d_image(image_data,target_shape=(64, 64, 64, 1))
        image_dict[j[0:6]] = image_data
        count += 1

#path1 = '/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/100206_T1w.nii.gz'

#print(shapes)
nifti_image = nib.load(path1)
image_data = nifti_image.get_fdata()
print("Shape of the NIfTI image:", image_data.shape)

slice_index = image_data.shape[2] // 2
slice_data = image_data[:, :, slice_index]

#plt.imshow(slice_data.T, cmap='gray', origin='lower')
#plt.title('Middle Slice of NIfTI Image')
#plt.axis('off')


Shape of the NIfTI image: (260, 311, 260)


In [21]:
import pandas as pd

df = pd.read_csv('/projectnb/ec523/projects/proj_GS_LQ_EPB/data/T1w/HCP/age/ages.csv')

ages = list(df['age'])
ids = list(df['subject_id'])
key = {}
for i,j in zip(ids,ages):
    key[i] = j

In [ ]:
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
input_shape = (64, 64, 64, 1)
#model.summary()
x_train = []
y_train = []
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for i in df['subject_id']:
    item = image_dict.get(str(i))
    if item is not None:
        x_train.append(item)
        y_train.append(key.get(i))
del image_dict

#print(y_train)
x_train = tf.convert_to_tensor(x_train, dtype=tf.float16)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float16)
lr = [1e-2, 1e-3, 1e-4]
beta_one = [.9, .99, .999]
beta_two = [.9, .99, .999]
#print(y_train)
count = 1
for fold, (train_idx, test_idx) in enumerate(kf.split(x_train)):
    for b1 in beta_one:
        for b2 in beta_two:
            for learn in lr:
                model_name = 'model' + str(count) + '_learn_' + str(learn) + '_beta1_' + str(b1) + '_beta2_' + str(b2)
                checkpoint = ModelCheckpoint(
                filepath= model_name + '.keras',       
                monitor='mae',            
                save_best_only=True,             
                mode='min',                     
                verbose=1                      
            )
                x_t = tf.gather(x_train,train_idx)
                x_val = tf.gather(x_train,test_idx)
                y_t = tf.gather(y_train,train_idx)
                y_val = tf.gather(y_train,test_idx)
                model = create_3d_cnn(input_shape,learning_rate=learn,beta1=b1,beta2=b2)
                history = model.fit(
                        x_t, y_t,
                        epochs=50,
                        batch_size=8,
                        validation_data=(x_val, y_val),
                        callbacks=[checkpoint]
                        )
                for i in range(len(history.history['loss'])):
                    train_loss = history.history['loss'][i]
                    val_loss = history.history['val_loss'][i]
                    #print(f"Epoch {i+1} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
                #SAVE MODEL HERE
                count += 1

Epoch 1/50


/usr4/ece601/lqushair/.conda/envs/EC523_model/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 93879.8359 - mae: 108.9537 
Epoch 1: mae improved from inf to 43.46453, saving model to model1_learn_0.01_beta1_0.9_beta2_0.9.h5


30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - loss: 91866.3984 - mae: 106.8412 - val_loss: 24.7927 - val_mae: 4.1843
Epoch 2/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 89.2646 - mae: 7.5130
Epoch 2: mae improved from 43.46453 to 7.11783, saving model to model1_learn_0.01_beta1_0.9_beta2_0.9.h5


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 88.2902 - mae: 7.4744 - val_loss: 21.0768 - val_mae: 3.9320
Epoch 3/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 59.9795 - mae: 6.4049
Epoch 3: mae improved from 7.11783 to 6.61872, saving model to model1_learn_0.01_beta1_0.9_beta2_0.9.h5


...

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 77.2158 - mae: 6.8324 - val_loss: 17.5014 - val_mae: 3.6112
Epoch 3/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 79.8157 - mae: 7.2416
Epoch 3: mae did not improve from 6.29866
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 78.7904 - mae: 7.1878 - val_loss: 64.5935 - val_mae: 7.1017
Epoch 4/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 60.9181 - mae: 6.2690
Epoch 4: mae improved from 6.29866 to 6.18101, saving model to model7_learn_0.01_beta1_0.9_beta2_0.999.h5


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 60.7531 - mae: 6.2579 - val_loss: 29.8489 - val_mae: 4.4644
Epoch 5/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 86.6630 - mae: 7.7307
Epoch 5: mae did not improve from 6.18101
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 86.9393 - mae: 7.7564 - val_loss: 67.9435 - val_mae: 7.3337
Epoch 6/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 66.6177 - mae: 6.3968
Epoch 6: mae did not improve from 6.18101
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 66.0683 - mae: 6.3836 - val_loss: 27.1662 - val_mae: 4.3226
Epoch 7/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 50.5135 - mae: 5.7849
Epoch 7: mae did not improve from 6.18101
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 51.2245 - mae: 5.8284 - val_loss: 17.5019 - val_mae: 3.6112
Epoch 8/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 61.1915 - mae: 6.4038
Epoch 8: mae did not improve from 6.18101
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 62.1217 - mae: 6.4421 - val_

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 56.8316 - mae: 6.3621 - val_loss: 32.4382 - val_mae: 4.5901
Epoch 12/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 59.7970 - mae: 6.3827
Epoch 12: mae did not improve from 5.88287
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 59.3550 - mae: 6.3435 - val_loss: 15.2240 - val_mae: 3.2926
Epoch 13/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 57.3017 - mae: 5.8384
Epoch 13: mae did not improve from 5.88287
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 58.0974 - mae: 5.8992 - val_loss: 25.7969 - val_mae: 4.0423
Epoch 14/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 66.7605 - mae: 6.4663
Epoch 14: mae did not improve from 5.88287
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 66.0188 - mae: 6.4187 - val_loss: 29.5018 - val_mae: 4.4467
Epoch 15/50
28/30 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 64.7465 - mae: 6.3854
Epoch 15: mae did not improve from 5.88287
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 64.2542 - mae: 6.367

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 37.0263 - mae: 4.8340 - val_loss: 26.2052 - val_mae: 4.2682
Epoch 17/50


KeyboardInterrupt: 